
## Steps

- 1st : `DONE` Scrap song name, author, hindi_text, translation text
- 2st : `DONE` convert hindi to iast 
- 3nd : `DONE` Create new song in db using above values
- 4rd : `DONE` get dictionary for greenmesg
- 5th : `DONE` create dic entry in dictionary.db
- 6th : `TODO` get line by line iast song, dict value
- 7th : `TODO` get line by line : synonyms
- 8th : `TODO` update song synonyms db

In [1]:
import os
from slokabase.SqliteModel import get_create_table_query, get_insert_query, get_read_query, get_update_query, get_delete_query
from slokabase.SqliteModel import SqliteModel
db_name = 'dictionary.db'
db_path = os.path.join(os.getcwd(),db_name)
DictonaryWord_sql = SqliteModel(db_path,'DictWord')
DictonaryMeaning_sql = SqliteModel(db_path,'DictMeaning')

db_name = 'slokabase.db'
db_path = os.path.join(os.getcwd(),db_name)
SongIndex_sql = SqliteModel(db_path,'SongIndex')
mySongs_sql = SqliteModel(db_path,'Songs')

In [9]:
# url = 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php'
# url = 'https://greenmesg.org/stotras/annapoorna/annapoorna_stotram.php'
url = 'https://greenmesg.org/stotras/vishnu/venkatesa_suprabhatam.php'

In [10]:
from slokabase.hindi2english import hindi2iast 
# hindi2iast(dict_entry[0]['hindi']).strip()
from bs4 import BeautifulSoup
import requests

In [13]:
source = requests.get(url).text
soup = BeautifulSoup(source)
song_name, song_author = get_name_author(soup)
hindi_text       = get_sanskrit_text(soup)
iast_text = hindi2iast(hindi_text)
translation_text = get_translation(soup)

In [1]:
# print(translation_text)
# print(iast_text)
# print(hindi_text)
print(len(hindi_text.split('\n\n')), len( translation_text.split('\n\n') )  )

NameError: name 'hindi_text' is not defined

In [6]:
# SongIndex_sql.create_entry(**song_info)
song_info = {}
song_info['song_name'] = song_name # 'My Dasavatara Stotram'
song_info['song_short_name'] = song_name.replace(" ","") # 'MyDasaVatStot'
song_info['sloka_statues'] = 1
song_info['division_no'] = 0
song_info['total_slokas'] = len(hindi_text.split('\n\n'))
# song_info['groups']
# song_info['devotion_god']
song_info['author']= song_author 
# song_info['describtion']=  
song_info['other_links '] = url



if len(SongIndex_sql.read_entry(**song_info)) !=0:
    # SongIndex_sql.create_entry(**song_info)
    song_idx = SongIndex_sql.read_entry( *['song_idx'], **song_info)[0]['song_idx'] # >>> 5
    print(SongIndex_sql.read_entry( *['song_idx'], **song_info))

    

[{'song_idx': 74}]


In [8]:
# song_info = dict()
# song_info['song_name'] = song_name # 'My Dasavatara Stotram'
# song_info['song_short_name'] = song_name.replace(" ","") # 'MyDasaVatStot'
# song_info['sloka_statues'] = 1
# song_info['division_no'] = 0
# song_info['total_slokas'] = len(hindi_text.split('\n\n'))
# # song_info['groups']
# # song_info['devotion_god']
# song_info['author']= song_author 
# # song_info['describtion']=  
# song_info['other_links '] = url

# song = dict()
# song['sloka_eng'] = iast_text
# song['sloka_hindi'] = hindi_text
# song['translation'] = translation_text




# mysong = dict()
# # mySongs_sql = SqliteModel(db_path,'Songs')
# mysong ['song_idx'] = SongIndex_sql.read_entry(*['song_idx'], **song_info)[0]['song_idx']

# list_sloka = song['sloka_eng'].split('\n\n')
# for idx, sloka in enumerate(list_sloka):
#     print(idx)
#     mysong['slokas_no']   = idx+1
#     mysong['sloka_eng']   = sloka.strip()
#     mySongs_sql.create_entry(**mysong)
# del mysong['sloka_eng']

# list_sloka = song['sloka_hindi'].split('\n\n')
# for idx, sloka in enumerate(list_sloka):
#     print(idx)
#     mysong['slokas_no']   = idx+1
#     mysong['sloka_hindi']   = sloka.strip()
#     mySongs_sql.update_entry( *['song_idx','slokas_no']  , **mysong)
# del mysong['sloka_hindi']

# list_sloka = song['translation'].split('\n\n')    
# for idx, sloka in enumerate(list_sloka):
#     print(idx)
#     mysong['slokas_no']   = idx+1
#     mysong['translation']   = sloka.strip()
#     mySongs_sql.update_entry( *['song_idx','slokas_no']  , **mysong)
# del mysong['translation']

In [12]:
def get_name_author(soup):
    container = soup.find("div", {"id": "Container"})
    song_name = container.h1.text.split('-')[0].strip() # 'Devi Aparadha Kshamapana Stotram'
    if '- composed by' in container.text: 
        song_author = container.text.split('- composed by')[1].split('\n')[0].strip() # 'Sri Adi Shankaracharya'
    else:
        song_author=None
    return song_name, song_author 

def get_sanskrit_text(soup):
    hindi_text = ''
    for verse in soup.select('span.Sanskrit'):
        for br in verse.find_all("br"):
            br.replace_with("\n")               
        hindi_text += verse.text
        hindi_text += '\n'
    
    hindi_text =hindi_text.strip()
    return hindi_text
    
def get_translation(soup):
    verse = soup.select('span.Sanskrit')[0]
    sub_containt = verse.parent#.text.split('||')
    for br in sub_containt.find_all("br"):
        br.replace_with("\n")
    output = ''
    for para in sub_containt.text.split('\n\n'):
        if 'Meaning' in para:
            # print('Meaning in para',para)
            lines = para.replace("Meaning:\n","").split('\n')
            for line in lines:
                split_list = line.split(':')
                prefix = split_list[0].strip()
                del split_list[0]
                text = '\n'.join(split_list)
                prefix_no = prefix.split('.')
                if len(prefix_no) ==2 and prefix_no[0].isnumeric() and prefix_no[1].isnumeric():
                    # print(text)
                    output +=text.strip() + '\n'
                else: 
                    # print(prefix,text) 
                    output += prefix + " " + text.strip() + '\n'
            # print()
            output +='\n'
    return output.strip()    

def scrap_word(main_word,sub_word_list):
    dict_entry= {} # dict()
    main_word = BeautifulSoup(main_word).text
    words_list = main_word.split("__")
    # print(words_list,main_word,sub_word_list)
    dict_entry['hindi'] = words_list[1].split('(')[0].strip()
    # p_iast_word = words_list[1].split('(')[1].split(')')[0].strip()
    dict_entry['sandhi']= words_list[1].split('(')[1].split(')')[0].strip()
    dict_entry['meaning'] = words_list[1].split('(')[1].split(')')[1].replace(':','').strip()
    # print(f'''{sanskrit_word}({p_iast_word}) = {meaning_word}''')
    if isinstance(sub_word_list, list):
        output_list = []
        for sub_word in sub_word_list:
            temp_dict = {} # dict()            
            temp_dict['hindi'] = BeautifulSoup( sub_word).text.split('(')[0]
            temp_dict['sandhi']=BeautifulSoup( sub_word).text.split('(')[1].split(')')[0].strip()
            temp_dict['meaning']=BeautifulSoup( sub_word).text.split('(')[1].split(')')[1].replace('=','').strip()
            # print()
            # print(f'''>>>{sanskrit_word}({p_iast_word}) = {meaning_word}''')
            output_list.append(temp_dict)
        dict_entry['subwords'] = output_list
    return dict_entry

def get_query_dic(temp_dict,url):
    dicword={}
    dicword['word'] =hindi2iast(temp_dict['hindi']).strip() # word
    dicword['hindi'] =temp_dict['hindi']                     # hindi 
    dicword['sandhi'] =temp_dict['sandhi']                    # sandhi
    if 'subwords' in temp_dict.keys() : 
        dicword['subwords'] = ''
        for dict in temp_dict['subwords']:
            dicword['subwords']+=hindi2iast(dict['hindi']).strip()+'-'
        dicword['subwords'] = dicword['subwords'][:-1]
    # print(dicword) 
    # # form sl
    dicmeaning = {}
    dicmeaning['word'] = dicword['word']
    dicmeaning['source'] = url
    dicmeaning['meaning_value'] = temp_dict['meaning']
    # print(dicmeaning)
    return dicword, dicmeaning
def create_dict_entry(DictonaryMeaning_sql, DictonaryWord_sql, dicword, dicmeaning):
    if len(DictonaryWord_sql.read_entry(word=dicword['word'])) ==1:
        if len(DictonaryWord_sql.read_entry(**dicword)) ==0:
            # print(f'word:{dicword["hindi"]} already exit but what about hindi, sandi,subword')
            DictonaryWord_sql.update_entry(*['word'],**dicword)
            # (**dicword)       
    else:
        DictonaryWord_sql.create_entry(**dicword)
    if len(DictonaryMeaning_sql.read_entry(word=dicmeaning['word'],meaning_value=dicmeaning['meaning_value'])) ==1:
        # already existing
        pass
    else : 
        last_idx = len(DictonaryMeaning_sql.read_entry(word=dicmeaning['word']))
        dicmeaning['meaning_idx'] = last_idx +1
        DictonaryMeaning_sql.create_entry(**dicmeaning)

def get_dic_list(line_list):
    dic_list=[]
    for idx, item in enumerate(line_list):
        if item == '' or item is None:
            continue
        if '<br/>' not in item:
            main_word = item
            sub_word =''
            dict_entry=scrap_word(main_word,sub_word)
            dic_list.append(dict_entry)
        else:
            all_words =item.split('<br/>')
            main_word = all_words[0]
            del all_words[0]
            sub_word_list = all_words
            dict_entry=scrap_word(main_word,sub_word_list)
            dic_list.append(dict_entry)
    return dic_list

# dic_list = get_dic_list(line_list)    

In [6]:
# # print
# print(get_name_author(soup))
# print(get_sanskrit_text(soup))
# print(get_translation(soup))

# curl 'https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0' -H 'Accept: */*' -H 'Accept-Language: en-US,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br' -H 'DNT: 1' -H 'Connection: keep-alive' -H 'Referer: https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php' -H 'Sec-Fetch-Dest: empty' -H 'Sec-Fetch-Mode: cors' -H 'Sec-Fetch-Site: same-origin'

In [7]:
# print(url)
# print(url.replace('stotras','dictionary/stotras').replace('php','txt'))
# print('https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt')

https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php
https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt
https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt


In [12]:
# x = requests.get('https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt')
# x =requests.request('GET','https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt')


In [10]:
# for item in line_list:
#     print(BeautifulSoup(item))


In [8]:
x =requests.request('GET', url.replace('stotras','dictionary/stotras').replace('php','txt') )
line_list = x.text.split('\r\n')
# for dict_entry in get_dic_list(line_list):
dict_entry = get_dic_list(line_list)[4]
print('dict_entry: ',dict_entry)
dicword={}
dicword['word'] =hindi2iast(dict_entry['hindi']).strip() # word
dicword['hindi'] =dict_entry['hindi']                     # hindi 
dicword['sandhi'] =dict_entry['sandhi']                    # sandhi
if 'subwords' in dict_entry.keys() : 
    dicword['subwords'] = ''
    for dict in dict_entry['subwords']:
        dicword['subwords']+=hindi2iast(dict['hindi']).strip()+'-'
    dicword['subwords'] = dicword['subwords'][:-1]
print('dicword: ',dicword) 
dicword, dicmeaning = get_query_dic(dict_entry,url)
print('get_query_dic=> dicword',dicword)
print('get_query_dic=> dicmeaning',dicmeaning)

dict_entry:  {'hindi': 'तदपि', 'sandhi': 'Tad-Api', 'meaning': 'Even that [I do not know]', 'subwords': [{'hindi': ' तत् ', 'sandhi': 'Tat', 'meaning': 'That'}, {'hindi': ' अपि ', 'sandhi': 'Api', 'meaning': 'Even'}]}
dicword:  {'word': 'tadapi', 'hindi': 'तदपि', 'sandhi': 'Tad-Api', 'subwords': 'tat-api'}
get_query_dic=> dicword {'word': 'tadapi', 'hindi': 'तदपि', 'sandhi': 'Tad-Api', 'subwords': 'tat-api'}
get_query_dic=> dicmeaning {'word': 'tadapi', 'source': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php', 'meaning_value': 'Even that [I do not know]'}


In [11]:
# dict_entry= [{'hindi': 'पापघ्नी', 'sandhi': 'Paapa-Ghnii', 'meaning': 'Remover of Sins', 
#  'subwords': [{'hindi': ' पाप ', 'sandhi': 'Paapa', 'meaning': 'Sin'}, 
#              {'hindi': ' घ्न ', 'sandhi': 'Ghna', 'meaning': 'Remover, Killer'}]},]
# # {'hindi': 'महादेवि', 'sandhi': 'Mahaadevi', 'meaning': 'Mahadevi, the great Goddess'}]
# temp_dict = dict_entry[0]



In [11]:
# dic_song_list = []
# for item in dic_list:
#     if 'subwords'  in item.keys() :
#         temp_list = []
#         for sub_item in item['subwords']:
#             temp_list.append((hindi2iast(sub_item['hindi']).strip(),sub_item['meaning'])) 
#         dic_song_list.append(temp_list)

#     dic_song_list.append((hindi2iast(item['hindi']).strip(),item['meaning']))        
# print(dic_song_list)

#### 1st Step: WebScrapping the dictionary form greenmesg

In [11]:
for dict_entry in get_dic_list(line_list):
    dicword, dicmeaning = get_query_dic(dict_entry,url)    
    create_dict_entry(DictonaryMeaning_sql, DictonaryWord_sql, dicword, dicmeaning)
    print('# mainword:',dicword, dicmeaning ) 
    if 'subwords' in dict_entry.keys() :     
        for dict_subword in dict_entry['subwords']:
            dicword, dicmeaning = get_query_dic(dict_subword,url)    
            create_dict_entry(DictonaryMeaning_sql, DictonaryWord_sql, dicword, dicmeaning)
            print('\t# subword:',dicword, dicmeaning)
            # print(dicword, dicmeaning ) 

# subword: {'word': 'tat', 'hindi': ' तत् ', 'sandhi': 'Tat'} {'word': 'tat', 'source': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php', 'meaning_value': 'That'}
# subword: {'word': 'api', 'hindi': ' अपि ', 'sandhi': 'Api'} {'word': 'api', 'source': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php', 'meaning_value': 'Even'}
# subword: {'word': 'stuti', 'hindi': ' स्तुति ', 'sandhi': 'Stuti'} {'word': 'stuti', 'source': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php', 'meaning_value': 'Praise, Eulogy'}
# subword: {'word': 'aho', 'hindi': ' अहो ', 'sandhi': 'Aho'} {'word': 'aho', 'source': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php', 'meaning_value': 'Alas'}
# subword: {'word': 'ca', 'hindi': ' च ', 'sandhi': 'Ca'} {'word': 'ca', 'source': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php', 'meaning_value': 'And, Also'}
# subword: {'word': 'āhvāna', 'hindi

In [7]:
meta_info = SongIndex_sql.read_entry(**song_info)[0]#['song_idx']
print(meta_info)
# song_idx = 
total_no_verse = meta_info['total_slokas']
# for sloka_no in range(1,total_no_verse+1)
sloka_no = 1 # 



{'song_idx': 74, 'song_name': 'Devi Aparadha Kshamapana Stotram', 'song_short_name': 'DeviAparadhaKshamapanaStotram', 'sloka_statues': 1, 'division': None, 'division_no': 0, 'total_slokas': 12, 'groups': None, 'devotion_god': None, 'author': 'Sri Adi Shankaracharya', 'describtion': None, 'other_links': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php'}


In [28]:
def get_synonyms_byverse(mySongs_sql,slokas_no,song_idx): # meta_info['song_idx']
    verse_line_list = mySongs_sql.read_entry(*['sloka_eng'],song_idx=song_idx,slokas_no=sloka_no)[0]['sloka_eng'].split('\n')
    # print(verse_line_list)
    if len(verse_line_list)==4:
        verse_line_list[1] =verse_line_list[1].split('|')[0].strip()
        verse_line_list[3] =verse_line_list[3].split('|')[0].strip()
    elif len(verse_line_list)==2:
        verse_line_list[0] =verse_line_list[0].split('|')[0].strip()
        verse_line_list[1] =verse_line_list[1].split('|')[0].strip()        
    # print(verse_line_list)
    output = ''
    for line_idx in verse_line_list:
        # for words in line_idx.split(" "):
        for word in line_idx.split(" "): #words:
            # word = words[4]
            # print(word)
            data = DictonaryWord_sql.read_entry(word=word)[0]
            # print(data)
            if isinstance(data['subwords'],str):
                for subword in data['subwords'].split('-'):
                    output += f"""{subword}={DictonaryMeaning_sql.read_entry(word=subword)[0]['meaning_value']}; """
                    # print(f"""{subword}={DictonaryMeaning_sql.read_entry(word=subword)[0]['meaning_value']};""", end=" ")
            output += f"""{data['word']}={DictonaryMeaning_sql.read_entry(word=data['word'])[0]['meaning_value']}; """ 
            # print(f"""{data['word']}={DictonaryMeaning_sql.read_entry(word=data['word'])[0]['meaning_value']};""", end =" ")
        output= output.strip() + '\n'
    return output.strip()#)
# get_synonyms_byverse(mySongs_sql,sloka_no,meta_info['song_idx'])   
get_synonyms_byverse(mySongs_sql,12,meta_info['song_idx'])   

'matsamaḥ=Like me; pātakī=Fallen; nāsti=Does not exist, Is not there; pāpa=Sin; ghna=Remover, Killer; pāpaghnī=Remover of Sins; tvatsamā=Like You; na=Not; hi=Indeed;\nevaṁ=Thus; jñātvā=Knowing; mahādēvi=Mahadevi, the great Goddess; yathā=As; yogya=Proper; yathāyogyaṁ=[Please do] whatever is proper; tathā=In like manner; kr̥=To Do; kuru=Please do;'

In [29]:
meta_info = SongIndex_sql.read_entry(**song_info)[0]#['song_idx']
print(meta_info)
total_no_verse = meta_info['total_slokas']
for sloka_no in range(1,total_no_verse+1):
# sloka_no = 1 # 
    print(sloka_no)
    if mySongs_sql.read_entry(song_idx=song_idx,slokas_no=sloka_no)[0]['synonyms'] is  None:
        synonyms_by_verse = get_synonyms_byverse(mySongs_sql,sloka_no,meta_info['song_idx'])
        mySongs_sql.update_entry( *['song_idx','slokas_no'],synonyms=synonyms_by_verse, song_idx=meta_info['song_idx'], slokas_no=sloka_no)

{'song_idx': 74, 'song_name': 'Devi Aparadha Kshamapana Stotram', 'song_short_name': 'DeviAparadhaKshamapanaStotram', 'sloka_statues': 1, 'division': None, 'division_no': 0, 'total_slokas': 12, 'groups': None, 'devotion_god': None, 'author': 'Sri Adi Shankaracharya', 'describtion': None, 'other_links': 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php'}
1
2
3
4
5
6
7
8
9
10
11
12


In [13]:
meta_info = SongIndex_sql.read_entry(**song_info)[0]#['song_idx']
total_no_verse = meta_info['total_slokas']
# for sloka_no in range(1,total_no_verse+1)
sloka_no = 1 # 


True